# 2. Outliers
In this notebook, we're going to make an overview on the dataframe's outliers. For that, we will apply the **Tukey's Fence** to detect them, since it's more robust to highly skewed datasets like this one, as we could check out in the preliminary EDA process.

Let's import the libraries and the data.

In [642]:
import numpy as np
import pandas as pd
import plotly.express as px

In [643]:
df = pd.read_csv("../data/data_formatted.csv")

In [644]:
df['fuel_type'].unique()

array(['Petrol', 'Electric', 'Diesel', 'Hybrid',
       'Plug-in Hybrid Petrol/Electric', 'Plug-in Hybrid',
       'Plug-in Hybrid Diesel/Electric', 'Petrol/Gas', 'Gas',
       'Petrol/Ethanol'], dtype=object)

In [645]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106149 entries, 0 to 106148
Data columns (total 27 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   fuel_type                    106149 non-null  object 
 1   mileage_km                   106149 non-null  float64
 2   body_type                    106149 non-null  object 
 3   transmission                 106149 non-null  object 
 4   power_hp                     104039 non-null  float64
 5   warranty_months              95528 non-null   float64
 6   emissions_label              97841 non-null   object 
 7   price                        106149 non-null  float64
 8   length_mm                    78242 non-null   float64
 9   width_mm                     78487 non-null   float64
 10  height_mm                    78234 non-null   float64
 11  wheelbase_mm                 78534 non-null   float64
 12  weight_kg                    78534 non-null   float64
 13 

Here, we define a function to plot the outliers and another one to get a dataframe with the total number of outliers in each variable, as well as the outliers ratio for each of them.

In [646]:
def plot_nans_outliers(data, x, y, nans_or_out, clr):

    fig = px.bar(
        data,
        x=x,
        y=y,
        title=f"Global Distribution of {nans_or_out}",
        color_discrete_sequence=[clr],
    )

    fig.update_layout(xaxis_tickangle=310)

    return fig

In [647]:
def outliers_tukey(df, k=1.5):

    numeric_data = (
        df.select_dtypes(include=np.number)
        .drop(
            columns=[
                "doors",
                "seats",
                "registration_month",
                "registration_year",
            ]
        )
        .columns.to_list()
    )

    df_outliers = pd.DataFrame(columns=["parameter", "normal_data", "outliers_%"])

    for col in numeric_data:
        q1 = np.nanquantile(df[col], q=0.25)
        q3 = np.nanquantile(df[col], q=0.75)

        ric = q3 - q1
        lim_inf = q1 - k * ric
        lim_sup = q3 + k * ric

        normal_data = df[df[col].between(lim_inf, lim_sup)].shape[0]
        total_outliers = df.shape[0] - normal_data
        per_outliers = round(total_outliers / df.shape[0] * 100, 2)

        df_outliers.loc[len(df_outliers)] = [col, total_outliers, per_outliers]

    df_outliers = df_outliers.sort_values(by="outliers_%", ascending=False).reset_index(
        drop=True
    )

    fig_out = plot_nans_outliers(
        df_outliers,
        x="parameter",
        y="outliers_%",
        nans_or_out="Outliers",
        clr="#2FE088",
    )

    return df_outliers, fig_out

Let's now get the df, the figure and plot it. There is a visualization page prepared in the Streamlit dashboard that can be consulted as well.

In [648]:
# Here we get the arranged df and the figure
df_out, fig_out = outliers_tukey(df)

In [649]:
df_out


,parameter,normal_data,outliers_%
0,cylinders,56876,53.58
1,co2_g_km,51893,48.89
2,engine_displacement_cm3,43033,40.54
3,warranty_months,36266,34.17
4,trunk_dim_2,35596,33.53
5,width_mm,34928,32.90
6,tank_capacity_l,34502,32.50
7,trunk_dim_1,32741,30.84
8,wheelbase_mm,32668,30.78
9,weight_kg,32265,30.40


In [650]:
fig_out

When we plot the outliers ratio per variable, we can se that there are many of them over a 20% of Outliers. Some of them, like the CO2 emmissions and the number of cylinders are over a 40%. We have to be careful now about how we're going to proceed.

The general plan is to analyze one by one or those of the same category (like dimensional variables) together and decide what to do.

A function is defined below to crop the df according to the Tukey's Fence.

In [651]:
def drop_outliers(data, cols, k = 1.5):

    for col in cols:
        q1 = np.nanquantile(data[col], q = 0.25)
        q3 = np.nanquantile(data[col], q = 0.75)

        ric = q3 - q1
        lim_inf = q1 - k*ric
        lim_sup = q3 + k*ric

        filtered_df = data[(data[col] >= lim_inf) & (data[col] <= lim_sup)]

    return filtered_df

For inputing values, wi will use the median depending on the body type, to ensure more accuracy.

In [652]:
data_medians = df.groupby(by='body_type')[df.select_dtypes(include='number').columns.to_list()].median()
data_medians

,mileage_km,power_hp,warranty_months,price,length_mm,width_mm,height_mm,wheelbase_mm,weight_kg,doors,...,cylinders,consumption_city_l_100km,consumption_highway_l_100km,consumption_mixed_l_100km,co2_g_km,tank_capacity_l,registration_month,registration_year,trunk_dim_1,trunk_dim_2
body_type,,,,,,,,,,,,,,,,,,,,,
Classic,10.0,NaN,36.0,34500.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,2023.0,NaN,NaN
Commercial,90812.0,102.0,12.0,13966.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2020.0,NaN,NaN
Convertible,49563.0,184.0,12.0,29990.0,4491.0,1810.0,1409.0,2605.0,1935.0,2.0,...,4.0,7.6,5.2,6.0,154.0,57.0,6.0,2018.0,190.0,215.000
Coupe,56005.0,313.0,12.0,50900.0,4673.0,1887.0,1404.0,2760.0,2065.0,2.0,...,6.0,7.5,5.3,6.2,170.0,66.0,6.0,2019.0,430.0,0.000
Coupe/Sports,27400.0,431.0,12.0,88429.5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2019.0,NaN,NaN
Estate,84859.0,143.0,12.0,19490.0,4668.0,1816.0,1459.0,2730.0,2025.0,5.0,...,4.0,5.2,3.9,4.4,115.0,53.0,6.0,2019.0,540.0,1.510
MPV,97200.0,130.0,12.0,16500.0,4419.0,1829.0,1645.0,2750.0,2080.0,5.0,...,4.0,5.4,4.2,4.7,119.0,55.0,6.0,2017.0,455.0,1.694
Pickup,160000.0,325.0,0.0,38900.0,4820.0,2063.0,1977.0,3119.0,3500.0,4.0,...,8.0,26.7,14.4,20.6,NaN,121.0,5.0,2005.0,623.0,1.492
Roadster,38889.5,306.0,12.0,52245.0,4376.5,1827.0,1291.0,2495.0,1739.0,2.0,...,6.0,11.4,6.4,8.2,214.0,64.0,6.0,2018.0,213.0,0.000


### mileage_km
In this column, there are just a few readings with over 1M kilometers. Those are probably wrong data and, given the low proportion of outliers her, we can safely remove them using the Tukey's fence with k=1.5.
Since the distribution is strongly skewed, we will apply a log transform before passing it to the ML model.

In [653]:
df = drop_outliers(df, ['mileage_km'])

In [654]:
df.shape

(102961, 27)

### price
This is the target and it contains important information. However, the strong outliers might alter the model results. These probably belong to the luxury segment. There are also values too low that shoul be corrected. 

The best way to proceed here is to apply a log transform later, so the model's performance won't be affected negatively, and remove the most upper extreme values applying Tukey with k=3.5 to remove the luxury segment. The lower values can be input with the median, depending on the boy type.

In [655]:
df[df['price'] < 1000].sort_values(by='price').head()

,fuel_type,mileage_km,body_type,transmission,power_hp,warranty_months,emissions_label,price,length_mm,width_mm,...,turbo,consumption_city_l_100km,consumption_highway_l_100km,consumption_mixed_l_100km,co2_g_km,tank_capacity_l,registration_month,registration_year,trunk_dim_1,trunk_dim_2
10028,Petrol,2.0,SUV,Automatic,129.0,12.0,C,1.0,4440.0,1863.0,...,Turbo,0.0,0.0,0.0,NaN,54.0,12,2023,452.0,0.00
20159,Diesel,180000.0,Coupe/Sports,Manual,231.0,0.0,NaN,200.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2008,NaN,NaN
39794,Diesel,67307.0,MPV,Manual,150.0,12.0,C,269.0,4796.0,1916.0,...,Turbo,5.6,4.6,5.0,129.0,70.0,7,2016,285.0,2.02
73536,Petrol,208000.0,Sedan,Manual,75.0,0.0,B,690.0,NaN,1652.0,...,Not specified,8.4,5.0,6.3,NaN,50.0,7,2003,0.0,0.00
72288,Diesel,170000.0,Sedan,Automatic,100.0,0.0,NaN,699.0,NaN,1798.0,...,Not specified,7.7,4.5,5.7,NaN,61.0,6,2004,0.0,0.00


In [656]:
# drop upper outliers
df = drop_outliers(df, ['price'], k=3.5)  

# input lower values
df['price'] = df.apply(
    lambda x: data_medians.loc[x['body_type'], 'price'] if x['price'] < 500 else x['price'],
    axis=1
)

In [657]:
df.shape

(98228, 27)

### power_hp
Similar situation here. Lower values, less than 60hp are unplausible, unless it's a motorcycle. Expensive models have much more power that regular ones. We're going to delete extreme values and apply a log transform to overcome the skewness.

In [658]:
df[df['power_hp'] < 5]

,fuel_type,mileage_km,body_type,transmission,power_hp,warranty_months,emissions_label,price,length_mm,width_mm,...,turbo,consumption_city_l_100km,consumption_highway_l_100km,consumption_mixed_l_100km,co2_g_km,tank_capacity_l,registration_month,registration_year,trunk_dim_1,trunk_dim_2
23323,Petrol,35000.0,SUV/Pickup,Automatic,3.000,NaN,C,68000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4,2021,NaN,NaN
44680,Hybrid,24192.0,SUV/Pickup,Automatic,1.000,12.0,ECO,37890.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,2022,NaN,NaN
44681,Hybrid,24192.0,SUV/Pickup,Automatic,1.000,12.0,ECO,37890.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2,2022,NaN,NaN
71796,Petrol,20101.0,SUV/Pickup,Manual,1.000,12.0,C,21290.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4,2022,NaN,NaN
71797,Petrol,20101.0,SUV/Pickup,Manual,1.000,12.0,C,21290.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4,2022,NaN,NaN
79202,Diesel,1.0,Small,Automatic,1.000,0.0,NaN,14500.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,2013,NaN,NaN
86067,Diesel,140000.0,Sedan,Manual,1.951,NaN,NaN,1200.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8,2001,NaN,NaN


In [659]:
# input lower values
df['power_hp'] = df.apply(
    lambda x: data_medians.loc[x['body_type'], 'power_hp'] if x['power_hp'] < 60 else x['power_hp'],
    axis=1
)

In [660]:
df = drop_outliers(df, ['power_hp'], k=3.5)

In [661]:
df.shape

(94220, 27)

### Dimensional variables
The next step will be to check the dimensional variables. The histograms look relatively normal in these variables:
- height_mm
- length_mm
- width_mm
- wheelbase_mm
- trunk dimensions 1 and 2
- weight_mm

Here, there is a remarkable percentage of outliers, so a good approach would be to imput the median value (less sensitive to outliers), taking into account the car type. It makes more sense that a sedan type has a similar height that other of the same type, instead of using a global median that would take into account the height of SUVs as well. This approach will only be applied at thos values that make no sense at all after checking the boxplots.

This, of course, won't remove all of them and it allows us to keep more consistent and realistic data that otherwise could impact negatively to our models.

Now we only have to input those values. Let's create a function to do it.

In [662]:
def input_outliers(data, input_data, cols, k = 1.5):

    for col in cols:
        q1 = np.nanquantile(data[col], q = 0.25)
        q3 = np.nanquantile(data[col], q = 0.75)

        ric = q3 - q1
        lim_inf = q1 - k*ric
        lim_sup = q3 + k*ric

        # inputs the values directly to the df
        data[col] = data.apply(
            lambda row: input_data.loc[row['body_type'], col] if (row[col] < lim_inf) or (row[col] > lim_sup) else row[col],
            axis=1
        )

    return data

Applying the function, we're going to be more permissive regarding the boundaries leaving k=2. This will just input very extreme values, specially those that look like errors, otherwise we would be dropping important information. 

The trunk dimensions and the wheelbase can be more dependent on the design. Let's adjust k=2.5 for them

Regarding the weight, we're going to input only the inferior part, since there are clear errors there (cars with 0 kg of weight).

In [663]:
df = input_outliers(df, data_medians, ['length_mm', 'width_mm', 'height_mm'], k=2)

In [664]:
df = input_outliers(df, data_medians, ['wheelbase_mm', 'trunk_dim_1', 'trunk_dim_2'], k=2.5)

In [665]:
df['weight_kg'] = df.apply(
    lambda row: data_medians.loc[row['body_type'], 'weight_kg'] if row['weight_kg'] < 900 else row['weight_kg'],
    axis=1
)

In [666]:
df.shape

(94220, 27)

### Consumption
This is a curious case, since there are many non-electric models with 0 consumption. Those values are going to be input with the median for each fuel type. Higher values, detected visualy, will be deleted. In the following lines we are not directly excluding the electric cars values, since they're gonna be input with the median of their type, which is zero.

In [667]:
data_cons = df.groupby(by='fuel_type')[['consumption_city_l_100km', 'consumption_highway_l_100km', 'consumption_mixed_l_100km']].median()
data_cons

,consumption_city_l_100km,consumption_highway_l_100km,consumption_mixed_l_100km
fuel_type,,,
Diesel,5.1,3.90,4.3
Electric,0.0,0.00,0.0
Gas,NaN,NaN,NaN
Hybrid,NaN,NaN,NaN
Petrol,6.0,4.20,4.9
Petrol/Ethanol,10.6,5.55,7.4
Petrol/Gas,6.5,4.50,5.2
Plug-in Hybrid,0.0,0.00,0.0
Plug-in Hybrid Diesel/Electric,3.8,4.10,4.0


In [668]:
cons_columns = ['consumption_city_l_100km', 'consumption_highway_l_100km', 'consumption_mixed_l_100km']

for col in cons_columns:
    df[col] = df.apply(
        lambda row: data_cons.loc[row['fuel_type'], col] if row[col] == 0 else row[col],
        axis=1
    )

Extreme consuptions, like over 20l/100km in mixed consumption are probably errorce or performance cars.

In [669]:
df = df[(df['consumption_city_l_100km'] < 25) | (df['consumption_city_l_100km'].isna())]
df = df[(df['consumption_highway_l_100km'] < 14) | (df['consumption_highway_l_100km'].isna())]
df = df[(df['consumption_mixed_l_100km'] < 20) | (df['consumption_mixed_l_100km'].isna())]

In [670]:
df.shape

(94212, 27)

### warranty_months
Here, we found several faulty values, like over 100 months of warranty. We're going to input the median directly to all cars above 50 months.

In [671]:
df.loc[df['warranty_months'] > 50, 'warranty_months'] = df['warranty_months'].median()

In [672]:
df.shape

(94212, 27)

### tank_capacity_l
Here, we must be careful, since the electric cars have a tank capacity of 0 liters. Excluding those, we can remove upper outliers (above 100l) and input the median to the lower ones (there are Sedans with 20-30 l of capacity, which doesn't really make sense unless it's a truck)

In [673]:
# input lower values excluding electric cars
df['tank_capacity_l'] = df.apply(
    lambda row: data_medians.loc[row['body_type'], 'tank_capacity_l'] if row['tank_capacity_l'] < 30 and row['fuel_type'] != 'Electric' else row['tank_capacity_l'],
    axis=1
)

# exclude upper extreme values that might be trucks
df = df[(df['tank_capacity_l'] < 100) | (df['co2_g_km'].isna())]

In [674]:
df.shape

(94128, 27)

### engine_displacement_cm3
This is a difficult one. The % of outliers is high and the distribution is very skewed. The values are realistic, so we won't delete anything for now, but let's consider applying log transform later.

### Emissions: co2_g_km
Also very skewed, so we can consider log transform. The upper fence is around 200 g/km. Above 500, these might be errors so we can a delete the upper values without affecting the lower ones (low emissions belong to the electric cars).

In [675]:
df = df[(df['co2_g_km'] < 400) | (df['co2_g_km'].isna())]

In [676]:
df.shape

(94128, 27)

### cylinder
Here, we probably have many outliers due to electric or mixedd-type cars. The higher values are for determined types of cars. So, for now, we're keeping them.

## Dataframe check
Now, let's chek out the dataframe after dealing with the outliers:

In [678]:
df.shape

(94128, 27)

In [679]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 94128 entries, 0 to 106148
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   fuel_type                    94128 non-null  object 
 1   mileage_km                   94128 non-null  float64
 2   body_type                    94128 non-null  object 
 3   transmission                 94128 non-null  object 
 4   power_hp                     94128 non-null  float64
 5   warranty_months              87647 non-null  float64
 6   emissions_label              87631 non-null  object 
 7   price                        94128 non-null  float64
 8   length_mm                    70369 non-null  float64
 9   width_mm                     70522 non-null  float64
 10  height_mm                    70358 non-null  float64
 11  wheelbase_mm                 70552 non-null  float64
 12  weight_kg                    70552 non-null  float64
 13  doors               

In [680]:
df.isna().sum()

fuel_type                          0
mileage_km                         0
body_type                          0
transmission                       0
power_hp                           0
warranty_months                 6481
emissions_label                 6497
price                              0
length_mm                      23759
width_mm                       23606
height_mm                      23770
wheelbase_mm                   23576
weight_kg                      23576
doors                          23577
seats                          23576
engine_displacement_cm3        23576
cylinders                      25591
turbo                          23576
consumption_city_l_100km       23576
consumption_highway_l_100km    23576
consumption_mixed_l_100km      23576
co2_g_km                       40422
tank_capacity_l                23576
registration_month                 0
registration_year                  0
trunk_dim_1                    23576
trunk_dim_2                    23576
d

So far, we haven't lose too much information. We still have to deal with NaN's, log transforms and the encodings in the next notebooks. Let's now save the outliers-free df to use it later.

In [681]:
df.to_csv('../data/data_outliers_free.csv', index=False)